In [ ]:
import numpy as np
from skimage import feature, color
from skimage.feature import hog, local_binary_pattern
from skimage import exposure
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

from keras.datasets import cifar10

In [ ]:
# Carrega o dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
# calcula lbp e hog
def LBP_HOG(X, combined_features):
    for image in X:
        gray_image = color.rgb2gray(image)  # transforma em preto e branco

        # lbp
        lbp = feature.local_binary_pattern(gray_image, n_points, radius, method="uniform")
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))

        # # cor utilizando a media e o desvio padrao
        # color_stats = np.hstack([np.mean(image, axis=(0, 1)), np.std(image, axis=(0, 1))])

        # hog
        hog_feature = hog(gray_image,
                          orientations=orientations,
                          pixels_per_cell=pixels_per_cell,
                          cells_per_block=cells_per_block,
                          block_norm='L2-Hys'
                        )

        # normalizacao
        lbp_hist = lbp_hist / (lbp_hist.sum() + 1e-6)
        hog_feature = exposure.rescale_intensity(hog_feature, in_range=(0, 10))

        # coloca na lista de features
        combined_feature = np.concatenate((hog_feature, lbp_hist))
        combined_features.append(combined_feature)


In [ ]:
# parametros
# hog
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# lbp
radius = 1
n_points = 8 * radius

In [ ]:
combined_features_train = []
combined_features_test = []

LBP_HOG(X_train, combined_features_train)
LBP_HOG(X_test, combined_features_test)

In [ ]:
# faz split para ter uma parte de validacao tambem
X_train, X_val, y_train, y_val = train_test_split(combined_features_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# treino
clf = RandomForestClassifier(n_estimators=300, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=0)

In [ ]:
# validacao
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("acurácia da validação:", accuracy)

acurácia da validação: 0.5304


In [ ]:
# teste
y_pred = clf.predict(combined_features_test)
accuracy = accuracy_score(y_test, y_pred)
print("acurácia do teste:", accuracy)

acurácia do teste: 0.5407


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

In [ ]:
import pandas as pd
results_dict = { "Model": 'LBP+HOG', "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1}

pd.DataFrame.from_dict([results_dict])

,Model,Accuracy,Precision,Recall,F1 Score
0,LBP+HOG,0.5407,0.540507,0.5407,0.535865
